In [2]:
import groio, rdkit
import numpy as np
from tqdm import tqdm
import sys
# sys.path.append('/home/kopanichuk/data/autopsist/utils')
from src.utils.io.gro import read_gro, write_gro
from src.utils.gro.Structure import Structure
from src.utils.gro.Atom import Atom
from src.utils.geom.AntiSphere import AntiSphere
from src.utils.geom.Sphere import Sphere
from src.utils.assembler.insert import insert_com_into_shape, find_position
from src.utils.assembler.push import push_atoms_apart

In [8]:
dir_gro = 'ff/gromos/gro/'

In [6]:
''' ПАРАМЕТРЫ НЕ ТРОГАТЬ!!! '''

insertion_limit = int(1e5)
rotation_limit = 10
package = 0.4
distance = {'min': 0.08**2, 'opt': 0.12**2}

In [9]:
system_size = np.full(3, 3.0) # nm
structure = Structure(box=system_size, atoms=[])
names = ['decane', 'spce']
density = dict(zip(names, [28/10, 33])) # nm-3
shapes = dict(zip(names, [
    Sphere(center=structure.get_center(), radius=1.0), 
    AntiSphere(center=structure.get_center(), radius=1.0, borders=structure.box)
]))
numbers = dict(zip(names, np.round([shapes[name].get_volume()*density[name] for name in names]).astype(int)))
print(numbers)
coms = []
for name in names:
    mol = read_gro(f'{dir_gro}{name}.gro').center_atoms_to_zero()
    mol_size = np.max(np.linalg.norm(mol.get_XYZ(), axis=1))    
    for mol_idx in tqdm(range(numbers[name])):
        new_com = insert_com_into_shape(
            insertion_limit, mol_idx, mol_size, 
            system_size, package, coms, shapes[name])
        coms.append(new_com)
        new_mol = find_position(rotation_limit, distance['min'], structure, new_com, mol)
        for atom in new_mol.atoms:
            new_atom = atom.copy()
            new_atom.idx = len(structure.atoms) + 1
            new_atom.mol_idx = len(coms) + 1
            structure.atoms.append(new_atom)

{'decane': 12, 'spce': 753}


100%|██████████| 753/753 [00:03<00:00, 227.59it/s] 


In [10]:
structure = push_atoms_apart(structure, distance['min'], distance['opt'])

Iteration 1


Atoms: 100%|██████████| 2379/2379 [01:05<00:00, 36.25it/s] 

0 overlaps detected


In [11]:
with open(f'na_diff/decwat/decane_water.gro', 'w') as f:
    f.write(write_gro(structure))